In [54]:
%matplotlib inline

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, MaxPooling2D
from keras.layers.convolutional import Conv2D 
from keras.layers.recurrent import SimpleRNN, LSTM, GRU 
from keras.utils import np_utils
from keras import backend as K

from distutils.version import LooseVersion as LV
from keras import __version__

from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

from keras.datasets import mnist, fashion_mnist, imdb

from sklearn.model_selection import train_test_split

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

print('Using Keras version:', __version__, 'backend:', K.backend())
assert(LV(__version__) >= LV("2.0.0"))

Using Keras version: 2.2.4 backend: tensorflow


In [55]:
if K.backend() == "tensorflow":
    import tensorflow as tf
    device_name = tf.test.gpu_device_name()
    if device_name == '':
        device_name = "None"
    print('Using TensorFlow version:', tf.__version__, ', GPU:', device_name)

Using TensorFlow version: 1.13.1 , GPU: /device:GPU:0


In [0]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [4]:
from google.colab import files
uploaded = files.upload()

Saving Churn_Modelling.csv to Churn_Modelling.csv


In [0]:
dataset=pd.read_csv('Churn_Modelling.csv')

In [58]:
dataset

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
5,6,15574012,Chu,645,Spain,Male,44,8,113755.78,2,1,0,149756.71,1
6,7,15592531,Bartlett,822,France,Male,50,7,0.00,2,1,1,10062.80,0
7,8,15656148,Obinna,376,Germany,Female,29,4,115046.74,4,1,0,119346.88,1
8,9,15792365,He,501,France,Male,44,4,142051.07,2,0,1,74940.50,0
9,10,15592389,H?,684,France,Male,27,2,134603.88,1,1,1,71725.73,0


In [0]:
#dependent and independent variables
X=dataset.iloc[:, 3:13].values
Y=dataset.iloc[:, 13].values

In [60]:
X

array([[619, 'France', 'Female', ..., 1, 1, 101348.88],
       [608, 'Spain', 'Female', ..., 0, 1, 112542.58],
       [502, 'France', 'Female', ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 'Female', ..., 0, 1, 42085.58],
       [772, 'Germany', 'Male', ..., 1, 0, 92888.52],
       [792, 'France', 'Female', ..., 1, 0, 38190.78]], dtype=object)

In [61]:
#Encoding categorical data
#Encoding the independent variable
from sklearn.preprocessing import LabelEncoder , OneHotEncoder
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])
onehotencoder = OneHotEncoder(categorical_features =[1])
X=onehotencoder.fit_transform(X).toarray()
X=X[:, 1:]

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:392: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


In [62]:
X

array([[0.0000000e+00, 0.0000000e+00, 6.1900000e+02, ..., 1.0000000e+00,
        1.0000000e+00, 1.0134888e+05],
       [0.0000000e+00, 1.0000000e+00, 6.0800000e+02, ..., 0.0000000e+00,
        1.0000000e+00, 1.1254258e+05],
       [0.0000000e+00, 0.0000000e+00, 5.0200000e+02, ..., 1.0000000e+00,
        0.0000000e+00, 1.1393157e+05],
       ...,
       [0.0000000e+00, 0.0000000e+00, 7.0900000e+02, ..., 0.0000000e+00,
        1.0000000e+00, 4.2085580e+04],
       [1.0000000e+00, 0.0000000e+00, 7.7200000e+02, ..., 1.0000000e+00,
        0.0000000e+00, 9.2888520e+04],
       [0.0000000e+00, 0.0000000e+00, 7.9200000e+02, ..., 1.0000000e+00,
        0.0000000e+00, 3.8190780e+04]])

In [0]:
#Cross validation by test and training data set
from sklearn.model_selection import train_test_split
X_train , X_test, Y_train , Y_test  = train_test_split(X,Y, test_size=0.2 , random_state=0)

In [0]:
#Feature scaling
from sklearn.preprocessing import StandardScaler
sc  =StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [65]:
X_train

array([[-0.5698444 ,  1.74309049,  0.16958176, ...,  0.64259497,
        -1.03227043,  1.10643166],
       [ 1.75486502, -0.57369368, -2.30455945, ...,  0.64259497,
         0.9687384 , -0.74866447],
       [-0.5698444 , -0.57369368, -1.19119591, ...,  0.64259497,
        -1.03227043,  1.48533467],
       ...,
       [-0.5698444 , -0.57369368,  0.9015152 , ...,  0.64259497,
        -1.03227043,  1.41231994],
       [-0.5698444 ,  1.74309049, -0.62420521, ...,  0.64259497,
         0.9687384 ,  0.84432121],
       [ 1.75486502, -0.57369368, -0.28401079, ...,  0.64259497,
        -1.03227043,  0.32472465]])

In [0]:
#make a ANN
#Initializing the ANN
classifier=Sequential()

In [0]:
 #Adding input layer and first hidden layer
classifier.add(Dense(activation='relu' , input_dim=11, units=6, kernel_initializer="uniform"))

In [0]:
#Adding second hidden layer
classifier.add(Dense(activation='relu',units=6, kernel_initializer="uniform"))

In [0]:
#Adding the output layer
classifier.add(Dense(activation="sigmoid",units=1, kernel_initializer="uniform"))

In [0]:
#compile the ANN
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [72]:
#Fitting the ANN in training set
classifier.fit(X_train,Y_train, batch_size=10, nb_epoch=100)

Epoch 1/100
 350/8000 [>.............................] - ETA: 3s - loss: 0.4193 - acc: 0.8229

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


8000/8000 [==============================] - 4s 440us/step - loss: 0.3991 - acc: 0.8349
Epoch 2/100
8000/8000 [==============================] - 3s 434us/step - loss: 0.3994 - acc: 0.8371
Epoch 3/100
8000/8000 [==============================] - 3s 432us/step - loss: 0.3999 - acc: 0.8349
Epoch 4/100
8000/8000 [==============================] - 3s 432us/step - loss: 0.3998 - acc: 0.8371
Epoch 5/100
8000/8000 [==============================] - 3s 433us/step - loss: 0.4000 - acc: 0.8341
Epoch 6/100
8000/8000 [==============================] - 3s 432us/step - loss: 0.3994 - acc: 0.8366
Epoch 7/100
8000/8000 [==============================] - 3s 432us/step - loss: 0.3997 - acc: 0.8356
Epoch 8/100
8000/8000 [==============================] - 3s 430us/step - loss: 0.3992 - acc: 0.8357
Epoch 9/100
8000/8000 [==============================] - 3s 434us/step - loss: 0.3995 - acc: 0.8346
Epoch 10/100
8000/8000 [==============================] - 3s 430us/step - loss: 0.3998 - acc: 0.8355
Epoch 11/10

In [0]:
#Predicting and evaluating the model
#predicting the test set results
Y_pred=classifier.predict(X_test)
Y_pred= (Y_pred>0.5)

In [76]:
Y_pred

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

In [0]:
#Making confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_test,Y_pred)


In [80]:
cm

array([[1543,   52],
       [ 255,  150]])

In [81]:
(1543+150)/2000

0.8465